In [73]:
import rospy
import tf2_ros
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, Vector3
from giskard_msgs.msg import WorldBody, CollisionEntry, WorldGoal, GiskardError
from giskardpy.python_interface.python_interface import GiskardWrapper
import math
from bayes_opt import BayesianOptimization
from giskardpy.motion_graph.tasks.task import WEIGHT_BELOW_CA



def reset_giskard():
    giskard.clear_motion_goals_and_monitors()
    giskard.world.clear()
    default_pose = {
        'r_elbow_flex_joint': -0.15,
        'r_forearm_roll_joint': 0,
        'r_shoulder_lift_joint': 0,
        'r_shoulder_pan_joint': 0,
        'r_upper_arm_roll_joint': 0,
        'r_wrist_flex_joint': -0.10001,
        'r_wrist_roll_joint': 0,
        'l_elbow_flex_joint': -0.15,
        'l_forearm_roll_joint': 0,
        'l_shoulder_lift_joint': 0,
        'l_shoulder_pan_joint': 0,
        'l_upper_arm_roll_joint': 0,
        'l_wrist_flex_joint': -0.10001,
        'l_wrist_roll_joint': 0,
        'torso_lift_joint': 0.2,
        'head_pan_joint': 0,
        'head_tilt_joint': 0,
        'l_gripper_l_finger_joint': 0.55,
        'r_gripper_l_finger_joint': 0.55
    }


    done = giskard.monitors.add_set_seed_configuration(default_pose)
    base_pose = PoseStamped()
    base_pose.header.frame_id = 'map'
    base_pose.pose.position = Point(0, 0, 0)
    base_pose.pose.orientation.w = 1
    done2 = giskard.monitors.add_set_seed_odometry(base_pose=base_pose)
    giskard.motion_goals.allow_all_collisions()
    giskard.monitors.add_end_motion(start_condition=f'{done} and {done2}')
    giskard.execute()
    giskard.clear_motion_goals_and_monitors()

In [5]:
rospy.init_node('giskard_examples')

In [6]:
giskard = GiskardWrapper()

In [92]:
reset_giskard()

# variables
single_joint_goal = {'torso_lift_joint': 0.3}
tool_frame = 'l_gripper_tool_frame'
cam_frame = 'wide_stereo_optical_frame'
base_link = 'base_footprint'
gripper_joint = 'l_gripper_l_finger_joint'
gripper_joint_open = 0.55
gripper_joint_close = 0.1
gripper_link = tool_frame

def giskard_sample(p1, p2, p3,):
    reset_giskard()
    # adding the object
    cyl_pose = PoseStamped()
    cyl_pose.header.frame_id = 'map'
    cyl_pose.pose.orientation.w = 1
    cyl_pose.pose.position = Point(2, -0.2, 1.2)
    giskard.world.add_cylinder(name='mycyl', height=0.25, radius=0.02, pose=cyl_pose, parent_link='map')

    obstacle_pose = PoseStamped()
    obstacle_pose.header.frame_id = 'mycyl'
    obstacle_pose.pose.orientation.w = 1
    obstacle_pose.pose.position = Point(-0.1, 0.1, 0)
    # giskard.world.add_box(name='obstacle', size=(0.02, 0.2, 0.1), pose=obstacle_pose, parent_link='mycyl')

    # define the robot features
    robot_pointing_feature = Vector3Stamped()
    robot_pointing_feature.header.frame_id = gripper_link
    robot_pointing_feature.vector = Vector3(1, 0, 0)

    robot_point_feature = PointStamped()
    robot_point_feature.header.frame_id = gripper_link
    robot_point_feature.point = Point(0, 0, 0)

    robot_gripper_z_axis_feature = Vector3Stamped()
    robot_gripper_z_axis_feature.header.frame_id = gripper_link
    robot_gripper_z_axis_feature.vector = Vector3(0, 0, 1)

    # define the world features
    world_cylinder_center_feature = PointStamped()
    world_cylinder_center_feature.header.frame_id = 'mycyl'
    world_cylinder_center_feature.point = Point(0, 0, 0)

    world_cyl_z_axis_feature = Vector3Stamped()
    world_cyl_z_axis_feature.header.frame_id = 'mycyl'
    world_cyl_z_axis_feature.vector = Vector3(0, 0, 1)

    # define the constraints
    giskard.motion_goals.add_distance(root_link='map',
                                      tip_link=gripper_link,
                                      reference_point=world_cylinder_center_feature,
                                      tip_point=robot_point_feature,
                                      lower_limit=0.01,
                                      upper_limit=0.01)
    mon_dist = giskard.monitors.add_distance(root_link='map',
                                             tip_link=gripper_link,
                                             reference_point=world_cylinder_center_feature,
                                             tip_point=robot_point_feature,
                                             lower_limit=0.01 - 0.005,
                                             upper_limit=0.01 + 0.005)

    giskard.motion_goals.add_height(root_link='map',
                                    tip_link=gripper_link,
                                    reference_point=world_cylinder_center_feature,
                                    tip_point=robot_point_feature,
                                    lower_limit=p1,
                                    upper_limit=p2)
    mon_height = giskard.monitors.add_height(root_link='map',
                                             tip_link=gripper_link,
                                             reference_point=world_cylinder_center_feature,
                                             tip_point=robot_point_feature,
                                             lower_limit=-0.09,
                                             upper_limit=0.09)
    giskard.motion_goals.add_pointing(root_link='map',
                                      tip_link=gripper_link,
                                      goal_point=world_cylinder_center_feature,
                                      pointing_axis=robot_pointing_feature)
    mon_pointing = giskard.monitors.add_pointing_at(root_link='map',
                                                    tip_link=gripper_link,
                                                    goal_point=world_cylinder_center_feature,
                                                    pointing_axis=robot_pointing_feature)

    giskard.motion_goals.add_align_planes(root_link='map',
                                          tip_link=gripper_link,
                                          goal_normal=world_cyl_z_axis_feature,
                                          tip_normal=robot_gripper_z_axis_feature)
    mon_align = giskard.monitors.add_vectors_aligned(root_link='map',
                                                     tip_link=gripper_link,
                                                     goal_normal=world_cyl_z_axis_feature,
                                                     tip_normal=robot_gripper_z_axis_feature)

    # giskard.motion_goals.allow_all_collisions()
    # giskard.motion_goals.avoid_collision(min_distance=p3, group1='obstacle')
    giskard.monitors.add_end_motion(start_condition=f'{mon_dist} and {mon_height} and {mon_align} and {mon_pointing}')
    giskard.monitors.add_cancel_motion(giskard.monitors.add_local_minimum_reached(),
                                       error_message='local minimum reached while monitors are not satisfied')
    result = giskard.execute()
    if result.error.code != GiskardError.SUCCESS:
        print(giskard.get_end_motion_reason(move_result=result, show_all=False))
        # tf_buffer = tf2_ros.Buffer()
        # tf_listener = tf2_ros.TransformListener(tf_buffer)
        # transform = tf_buffer.lookup_transform('l_gripper_tool_frame', 'mycyl', rospy.Time(0), rospy.Duration(1.0))
        # return -math.sqrt(transform.transform.translation.x**2 + transform.transform.translation.y**2)
        return 0
    else:
        return 1



In [93]:
# Bounded region of parameter space
pbounds = {'p1': (-0.08, 0), 'p2': (0.01, 0.08), 'p3': (0.01, 0.1)}

optimizer = BayesianOptimization(
    f=giskard_sample,
    pbounds=pbounds,
    random_state=378,
)

In [94]:
optimizer.maximize(
    init_points=3,
    n_iter=3,
)

|   iter    |  target   |    p1     |    p2     |    p3     |
-------------------------------------------------------------
{'PointingAt': False}
| 1         | 0.0       | -0.03245  | 0.01357   | 0.01451   |
{'PointingAt': False}
| 2         | 0.0       | -0.01469  | 0.03815   | 0.01358   |
{'PointingAt': False}
| 3         | 0.0       | -0.07842  | 0.01674   | 0.07861   |
| 4         | 1.0       | -0.000687 | 0.07616   | 0.09925   |
{'PointingAt': False}
| 5         | 0.0       | -0.06557  | 0.05318   | 0.08829   |
{'PointingAt': False}
| 6         | 0.0       | -0.05707  | 0.07648   | 0.01368   |


In [62]:
print(optimizer.max)

{'target': -0.0007272502698663801, 'params': {'p1': -0.09998856251826552, 'p2': 0.030233257263183978}}


In [56]:
giskard_sample(-0.05, 0.05)

{'DistanceMonitor': False, 'PointingAt': False}


0.1885551856693502

In [ ]:
%%bash
pip3 install bayesian_optimization==1.4.0
pip3 install colorama==0.4.4